In [1]:
def createLabel(fname, seq_len):
    print("Creating label . . .")
    filename = fname.split('/')
    removeOutput("{}_label_{}.txt".format(filename[1][:-4], seq_len))
    
    df = pd.read_csv(fname, parse_dates = True, index_col = 0)
    df.fillna(0)
    
    df.reset_index(inplace = True)
    df['Date'] = df['Date'].map(mdates.date2num)
    for i in range(0, len(df)):
        c = df.ix[i:i + int(seq_len), :]
        
        starting = 0
        endvalue = 0
        label = ""
        if len(c) == int(seq_len) + 1:
            starting = c["Open"].iloc[-1]
            endvalue = c["Close"].iloc[-1]
            tmp_rtn = endvalue / starting - 1
            if tmp_rtn > 0:
                label = 1
            else:
                label = 0
            with open("{}_label_{}.txt".format(filename[1][:-4], seq_len), 'a') as the_file:
                the_file.write("{}-{},{}".format(filename[1][:-4], i, label))
                the_file.write("\n")
    print("Create label finished.")

In [3]:
def ohlc2cs(fname, seq_len, dataset_type, dimension, use_volume):
    print("Converting ohlc to candlestick")
    symbol = fname.split('_')[0]
    symbol = symbol.split('/')[1]
    print(symbol)
    path = "{}".format(os.getcwd())
    if not os.path.exists("{}/dataset/{}_{}/{}/{}".format(path, seq_len, dimension,
                                                         symbol, dataset_type)):
            os.makedirs("{}/dataset/{}_{}/{}/{}".format(path, seq_len, dimension, symbol,
                                                       dataset_type))
        
    df = pd.read_csv(fname, parse_dates=True, index_col = 0)
    df.fillna(0)
    
    plt.style.use('dark_background')
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].map(mdates.date2num)
    for i in range(0, len(df)-int(seq_len)):
        c = df.ix[i:i + int(seq_len) - 1, :]
        if len(c) == int(seq_len):
            my_dpi = 96
            fig = plt.figure(figsize=(dimension / my_dpi,
                                     dimension / my_dpi), dpi=my_dpi)
            ax1 = fig.add_subplot(1, 1, 1)
            candlestick2_ochl(ax1, c['Open'], c['Close'], c['High'], c['Low'],
                             width = 1, colorup = '#77d879', colordown='#db3f3f')
            ax1.grid(False)
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.xaxis.set_visible(False)
            ax1.yaxis.set_visible(False)
            ax1.axis('off')
            
            if use_volume:
                ax2 = ax1.twinx()
                bc = volume_overlay(ax2, c['Open'], c['Close'], c['Volume'],
                                   colorup = '#77d879', colordown = '#db3f3f',
                                   alpha = 0.5, width = 1)
                ax2.add_collection(bc)
                ax2.grid(False)
                ax2.set_xticklabels([])
                ax2.set_yticklabels([])
                ax2.xaxis.set_visible(False)
                ax2.yaxis.set_visible(False)
                ax2.axis('off')
            pngfile = 'dataset/{}_{}/{}/{}/{}-{}.png'.format(
            seq_len, dimension, symbol, dataset_type, fname[11:-4], i)
            fig.savefig(pngfile, pad_inches=0, transparent=False)
            plt.close(fig)
    
    print("Converting olhc to candlestik finished.")

In [4]:
def image2dataset(input, label_file):
    label_dict = {}
    with open(label_file) as f:
        for line in f:
            (key, val) = line.split(',')
            label_dict[key] = val.rstrip()
    path = "{}/{}".format(os.getcwd(), input)
    for filename in os.listdir(path):
        if filename is not '':
            for k, v in label_dict.items():
                splitname = filename.split("_")
                f, e = os.path.splitext(filename)
                newname = "{}_{}".format(splitname[0], splitname[1])
                if newname == k:
                    new_name = "{}{}.png".format(v, f)
                    os.rename("{}_{}".format(path, filename),
                             "{}/{}".format(path, new_name))
                    break
    folders = ['1', '0']
    for folder in folders:
        if not os.path.exists("{}/classes/{}".format(path, folder)):
            os.makedirs("{}/classes/{}".format(path, folder))
            
    for filename in os.listdir(path):
        if filename is not '' and filename is not 'classes':
            f, e = os.path.splitext(filename)
            if label_dict[f] == '1':
                move("{}/{}".format(path, filename),
                    "{}/classes/1/{}".format(path, filename))
            elif label_dict[f] == "0":
                move("{}/{}".format(path, filename),
                    "{}/classes/0/{}".format(path, filename))
    print('Done')

In [10]:
import os
import sys
from shutil import copyfile
pathdir = sys.argv[1]
origindir = sys.argv[2]
targetdir = sys.argv[3]

counttest = 0
counttrain = 0
for root, dirs, files in os.walk("{}/{}".format(pathdir, origindir)):
    
    for file in files:
        tmp = root.replace('\\', '/')
        tmp_label = tmp.split('/')[-1]
        
        if tmp_label == '0':
            if 'test' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/test/0/{}".format(
                pathdir, targetdir, file)
                copyfile(origin, destination)
                counttest += 1
            elif 'train' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/train/0/{}".format(
                pathdir, targetdir, file)
                copyfile(origin, destination)
                counttrain += 1
        elif tmp_label == '1':
            if 'test' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/test/1/{}".format(
                pathdir, targetdir, file)
                copyfile(origin, destination)
                counttest += 1
            elif 'train' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/train/1/{}".format(
                pathdir, targetdir, file)
                copyfile(origin, destination)
                counttrain += 1

IndexError: list index out of range

In [ ]:
def build_model(SHAPE, nb_classes, bn_axis, seed=None):
    if seed:
        np.random.seed(seed)
    input_layer = Input(shape=SHAPE)
    
    x = Conv2D(32, 3, 3, init='glorot_uniform')